In [ ]:
import open3d as o3d
import numpy as np
import random

def generate_superpoints(pcd, voxel_size=0.02, min_points=30):
    """
    基于体素下采样+区域生长分割，生成超点（superpoints）。
    """
    down_pcd = pcd.voxel_down_sample(voxel_size=voxel_size)
    down_pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2.0, max_nn=30)
    )

    labels = np.array(
        down_pcd.cluster_dbscan(eps=voxel_size * 1.5, min_points=min_points, print_progress=True)
    )

    if labels.max() == -1:
        raise ValueError("DBSCAN未找到任何有效超点！请调整 voxel_size 或 min_points 参数。")

    print(f"生成了 {labels.max() + 1} 个超点区域")
    return down_pcd, labels

def colorize_superpoints(pcd, labels):
    """
    给不同的超点（标签）随机赋色。
    """
    max_label = labels.max()
    if max_label == -1:
        raise ValueError("无法上色：超点标签全为-1（噪声）")

    # 生成颜色
    colors = np.array([np.random.rand(3) for _ in range(max_label + 1)])

    # 给未分类的噪声点指定黑色
    colors_with_noise = np.vstack([colors, np.array([[0, 0, 0]])])

    # 注意：labels中-1要映射到最后一行（黑色）
    mapped_labels = np.where(labels == -1, max_label + 1, labels)

    pcd.colors = o3d.utility.Vector3dVector(colors_with_noise[mapped_labels])
    return pcd

def save_colored_pcd(pcd, output_filename):
    """
    保存着色后的点云。
    """
    o3d.io.write_point_cloud(output_filename, pcd)
    print(f"保存着色后的点云到：{output_filename}")


# 加载原始点云
input_filename = "/hpc2hdd/home/qyao951/sam2/SAM3D/crop_pcd.ply"  # 这里替换成你的点云文件
pcd = o3d.io.read_point_cloud(input_filename)
print(f"原始点云包含 {len(pcd.points)} 个点")

# 生成超点
down_pcd, labels = generate_superpoints(pcd, voxel_size=0.002, min_points=30)

# 给超点上色
colored_pcd = colorize_superpoints(down_pcd, labels)

# 保存新的点云
save_colored_pcd(colored_pcd, "colored_superpoints.ply")

# 可视化
# o3d.visualization.draw_geometries([colored_pcd])


原始点云包含 206098 个点
